4-1 計算環境:  Google Colaboratory

メリット
    - 環境構築に手間がかからない  
    - ライブラリが入っている
    - GPU計算が簡単
    

利用するライブラリ

- transformaers  
    ニューラル言語モデルのライブラリ  
- Fugashi  
    MeCabのPython用ライブラリ  
- ipadic  
    MeCabで形態素解析を行う際に用いる辞書  
    
Colaboratory では, !の後にコマンドを書くとコマンド実行できる







In [1]:
!pip install transformers fugashi ipadic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 36.4 MB/s 
     |████████████████████████████████| 568 kB 72.4 MB/s 
     |████████████████████████████████| 13.4 MB 30.3 MB/s 
     |████████████████████████████████| 101 kB 11.2 MB/s 
     |████████████████████████████████| 6.6 MB 59.7 MB/s 
     |████████████████████████████████| 596 kB 67.1 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=47cadb74db9b4e6976dccad2a99fd020d4158d2f2ba707166c313cc9e01e139c
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
Successfully built ipadic
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
import torch 
from transformers import BertJapaneseTokenizer, BertModel


- Transformers  
    - Huggingface 社が提供しているN言語モデルのOSS  
    - cl-touhoku/bert-base-japanese-whole-word-masking  
        - 東北大学がWikipediaの日本語記事データを用いて学習したモデル  
        
    - 処理手順  
    1. トークナイザを用いて,文章をトークン化
    2. 処理したデータをBERTに入力し,出力を得る  
    
- トークナイザ  
    文章をトークンに分割し,BERTに入力できる形に変換.

In [7]:
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
# 学習済みのトークナイザをロード
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)

内部では, 
1. MeCabで単語分割  
2. WOrdPieceでトークン化  
している.

In [8]:
#トークン化してみる
tokenizer.tokenize('明日は自然言語処理の勉強をしよう.')

['明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う', '.']

In [9]:
tokenizer.tokenize('明日はマシンラーニングの勉強をしよう.')

['明日', 'は', 'マシン', '##ラー', '##ニング', 'の', '勉強', 'を', 'しよ', 'う', '.']

｢##｣はサブワード分割の結果,最初の単語以外につく.

In [10]:
tokenizer.tokenize('機械学習を中国語にすると机器学习だ.')

['機械', '学習', 'を', '中国', '語', 'に', 'する', 'と', '机', '器', '学', '[UNK]', 'だ', '.']

习が未知語[UNK]トークンに変換されている.

In [12]:
#文章を符号化(ID化)するには encode()を使う.
input_ids = tokenizer.encode('明日は自然言語処理の勉強をしよう.')
print(input_ids)


[2, 11475, 9, 1757, 1882, 2762, 5, 8192, 11, 2132, 205, 143, 3]


encode()は,  
- トークン列の先頭に[CLS]  
- 末尾に[SEP]  
を自動で追加する.


In [13]:
# ID ー> トークン は convert_ids_to_tokens()
tokenizer.convert_ids_to_tokens(input_ids)


['[CLS]', '明日', 'は', '自然', '言語', '処理', 'の', '勉強', 'を', 'しよ', 'う', '.', '[SEP]']

In [ ]:
#BERTを利用するためにトークン列の長さ(系列長)を同じに揃える
#系列長が短ければ特殊トークン[PAD]を末尾に足す
#ながければ末尾トークンを削る
#[PAD]部分は処理に関係ないので,
#関係ある部分のみを表すattenntion_maskを用意しておく

# tokenizerで指定した系列帳に揃えられたidやmaskの辞書が手に入る
# max_length : 系列長
# padding='max_length': 足りなかったら12まで[PAD]を足す
# truncation: 多かったら削る

text = '明日の天気は晴れだ.'
encoding = tokenizer(
    text, max_length=12, padding='max_length', truncation=True
)
print('# encoding:')
print(encoding)

tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
print('# tokens:')
print(tokens)

# encoding:
{'input_ids': [2, 11475, 5, 11385, 9, 16577, 75, 143, 3, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]}
# tokens:
['[CLS]', '明日', 'の', '天気', 'は', '晴れ', 'だ', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]']


In [ ]:
#系列長を短くしてみる
encoding = tokenizer(
    text, max_length=6, padding='max_length', truncation=True
)
print('# encoding:')
print(encoding)

tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
print('# tokens:')
print(tokens)

# encoding:
{'input_ids': [2, 11475, 5, 11385, 9, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}
# tokens:
['[CLS]', '明日', 'の', '天気', 'は', '[SEP]']


In [ ]:
#truncationをFalseにしてみる
encoding = tokenizer(
    text, max_length=6, padding='max_length', truncation=False
)
print('# encoding:')
print(encoding)

tokens = tokenizer.convert_ids_to_tokens(encoding['input_ids'])
print('# tokens:')
print(tokens)

# encoding:
{'input_ids': [2, 11475, 5, 11385, 9, 16577, 75, 143, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}
# tokens:
['[CLS]', '明日', 'の', '天気', 'は', '晴れ', 'だ', '.', '[SEP]']


In [ ]:
#復数の文章をリストで渡せる
text_list = [ '明日の天気は晴れだ.'
             ,'パソコンが急に動かなくなった.' ]

#それぞれリストが返ってくる
# padding=longest  で最も長い文章が系列長になる

# BERTに入力する場合は数値配列はPyTorchの多次元配列を扱うための型
# torch.Tensorに変換する必要がある
# return_tensors='pt'でテンソルとして出力されそのまま入力可能
encoding = tokenizer( text_list
                     , max_length = 10
                     , padding='max_length'
                     , truncation=True
                     , return_tensors='pt'
)
print('# encoding:')
print(encoding)

# encoding:
{'input_ids': tensor([[    2, 11475,     5, 11385,     9, 16577,    75,   143,     3,     0],
        [    2,  6311,    14,  1132,     7, 16084,   332,    58,    10,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [4]:
#符号化(トークン化→ID化)されたデータをBERTに入力する
#モデルのロード
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
bert = BertModel.from_pretrained(model_name)

torch.device('mps')

#BERTをGPUに載せる
bert = bert.cuda()

#モデルの概要
print(bert.config)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertConfig {
  "_name_or_path": "cl-tohoku/bert-base-japanese-whole-word-masking",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertJapaneseTokenizer",
  "transformers_version": "4.21.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}



In [17]:
text_list = [ '明日は自然言語処理の勉強をしよう.'
            , '明日はマシーンラーニングの勉強をしよう.']

encoding = tokenizer(  text_list
                     , max_length=32
                     , padding='max_length'
                     , truncation=True
                     , return_tensors='pt'
)

#データをGPUに載せる
encoding = {k:v.cuda() for k, v in encoding.items()}

# BERTでの処理処理 
# ｢**｣はencodingの中身を展開して入力
output = bert(**encoding) #それぞれの入力は2次元のtorch.Tensor

###
# 記述としては以下と同じ
output = bert(  input_ids = encoding['input_ids']
              , attention_mask=encoding['attention_mask']
              , token_type_ids=encoding['token_type_ids']
)
###


# 最終層の出力(いわゆるモデルのモデルのOutput)
last_hidden_state = output.last_hidden_state

#テンソルのサイズ
# 左から順にパッチサイズ2, 系列長32, 隠れ状態の次元768
print(last_hidden_state.size())

torch.Size([2, 32, 768])
